# 様々な環境での推論: ONNX と Menoh の活用

これまでの章では、Chainer を用いた学習済みモデルの作成方法・推論の実行方法を解説しました。

しかし、実務においては Chainer で作成した学習済みモデルをそのまま推論に用いることが難しい場面もあります。
例えば、Ruby や Java など Python 以外の言語で実装された既存の Web アプリケーションに推論機能を組み込みたい場合や、モバイルデバイス上で推論を行いたい場合、Chainer（Python）で実装された学習済みモデルを直接実行するのは困難です。

そこで本章では、Chainer の学習済みモデルを ONNX（オニキス）と呼ばれる形式に変換することで、異なるフレームワークを使用して推論を実行する方法について解説します。
また、Menoh（メノウ）というフレームワークを使用して、ONNX 形式の学習済みモデルを様々なプログラミング言語で推論する方法もご紹介します。

## ONNX とは？

ONNX は **Open Neural Network eXchange** の略で、ディープラーニングの学習済みモデルを様々なフレームワーク間で相互に交換するためのファイルフォーマット（形式）です。

学習済みモデルを ONNX 形式に変換することで、あるフレームワークで訓練したモデルを別のフレームワークでの推論に使用できるようになります。また、代表的な手法の学習済みモデルは [ONNX Model Zoo](https://github.com/onnx/models) で公開されており、ダウンロードしてそのまま使用することも可能です。

### ONNX のサポート状況

数多くのディープラーニングフレームワークが ONNX をサポートしています。
最新の対応状況は [ONNX の GitHub ページ](https://github.com/onnx/tutorials) から確認することができます。
2019年2月現在のサポート状況は下記の通りです。

<!--
表の作成手順:

1. https://raw.githubusercontent.com/onnx/tutorials/989599017f3eb9846a4dd593b85427babf233eef/README.md からテーブルをコピー
2. 本章で使用する Chainer, Menoh, Caffe2 を先頭に移動
3. ヘッダ行を日本語化
   (Framework / tool | Installation | Exporting to ONNX (frontend) | Importing ONNX models (backend))
-->

| フレームワーク/ツール | インストール方法 | エクスポート | インポート |
| --- | --- | --- | --- |
| [Chainer](https://chainer.org/) | [chainer/onnx-chainer](https://github.com/chainer/onnx-chainer) | [Exporting](tutorials/ChainerOnnxExport.ipynb) | coming soon |
| [Menoh](https://github.com/pfnet-research/menoh) | [pfnet-research/menoh](https://github.com/pfnet-research/menoh) | n/a | [Importing](tutorials/OnnxMenohHaskellImport.ipynb) |
| [Caffe2](http://caffe2.ai) | [part of caffe2 package](https://github.com/pytorch/pytorch/tree/master/caffe2/python/onnx) | [Exporting](tutorials/Caffe2OnnxExport.ipynb) | [Importing](tutorials/OnnxCaffe2Import.ipynb) |
| [PyTorch](http://pytorch.org/) | [part of pytorch package](http://pytorch.org/docs/master/onnx.html) | [Exporting](tutorials/PytorchOnnxExport.ipynb), [Extending support](tutorials/PytorchAddExportSupport.md) | coming soon |
| [Cognitive Toolkit (CNTK)](https://www.microsoft.com/en-us/cognitive-toolkit/) | [built-in](https://docs.microsoft.com/en-us/cognitive-toolkit/setup-cntk-on-your-machine) | [Exporting](tutorials/CntkOnnxExport.ipynb) | [Importing](tutorials/OnnxCntkImport.ipynb) |
| [Apache MXNet](http://mxnet.incubator.apache.org/) | part of mxnet package [docs](http://mxnet.incubator.apache.org/api/python/contrib/onnx.html) [github](https://github.com/apache/incubator-mxnet/tree/master/python/mxnet/contrib/onnx) | [Exporting](tutorials/MXNetONNXExport.ipynb) | [Importing](tutorials/OnnxMxnetImport.ipynb) |
| [TensorFlow](https://www.tensorflow.org/) | [onnx/onnx-tensorflow](https://github.com/onnx/onnx-tensorflow) and [onnx/tensorflow-onnx](https://github.com/onnx/tensorflow-onnx) | [Exporting](tutorials/OnnxTensorflowExport.ipynb) | [Importing](tutorials/OnnxTensorflowImport.ipynb) [experimental] |
| [Apple CoreML](https://developer.apple.com/documentation/coreml) | [onnx/onnx-coreml](https://github.com/onnx/onnx-coreml) and [onnx/onnxmltools](https://github.com/onnx/onnxmltools) | [Exporting](https://github.com/onnx/onnxmltools) | [Importing](tutorials/OnnxCoremlImport.ipynb) |
| [SciKit-Learn](http://scikit-learn.org/) | [onnx/onnxmltools](https://github.com/onnx/onnxmltools) | [Exporting](https://github.com/onnx/onnxmltools) | n/a |
| [ML.NET](https://github.com/dotnet/machinelearning/) | [built-in](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.models.onnxconverter.convert?view=ml-dotnet#definition) | [Exporting](https://github.com/dotnet/machinelearning/blob/master/test/Microsoft.ML.Tests/OnnxTests.cs) | [Importing](https://github.com/dotnet/machinelearning/blob/master/test/Microsoft.ML.OnnxTransformTest/OnnxTransformTests.cs#L186) |
| [MATLAB](https://www.mathworks.com/) | [onnx converter on matlab central file exchange](https://www.mathworks.com/matlabcentral/fileexchange/67296) | [Exporting](https://www.mathworks.com/help/deeplearning/ref/exportonnxnetwork.html) | [Importing](https://www.mathworks.com/help/deeplearning/ref/importonnxnetwork.html) |
| [TensorRT](https://developer.nvidia.com/tensorrt) | [onnx/onnx-tensorrt](https://github.com/onnx/onnx-tensorrt) | n/a | [Importing](https://github.com/onnx/onnx-tensorrt/blob/master/README.md) |

エクスポートとは、そのフレームワークで作成した学習済みモデルを ONNX 形式に変換できることを表します。
また、インポートとは、そのフレームワークが ONNX 形式の学習済みモデルを読み込んで推論を実行できることを表します。

Chainer は、現時点ではエクスポートのみに対応しています。
インポートについても今後サポートが予定されています。

なお、学習済みモデルで使用している関数が ONNX 形式でサポートされていない場合は、エクスポートすることができません。
サポートされている関数の一覧は [ONNX の公式ドキュメント](https://github.com/onnx/onnx/blob/master/docs/Operators.md)で確認することができます。
また、ONNX 形式でサポートされている関数であっても、フレームワークによってはエクスポート/インポートに制約がある場合があります。
詳しくは各フレームワークの ONNX に関するドキュメント（例: [Chainer](https://github.com/chainer/onnx-chainer#supported-functions)、[PyTorch](https://pytorch.org/docs/stable/onnx.html#supported-operators)）を確認してください。

## Chainer の学習済みモデルを ONNX 形式に変換する

Chainer の学習済みモデルを ONNX 形式に変換する方法を解説します。

今回は Chainer にデフォルトで用意されている、VGG16 の学習済みモデルを使用します。

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L

In [20]:
# VGG16 モデルの読み込み
model = L.VGG16Layers()

In [21]:
# VGG16 のアーキテクチャーの確認
model.available_layers

['conv1_1',
 'conv1_2',
 'pool1',
 'conv2_1',
 'conv2_2',
 'pool2',
 'conv3_1',
 'conv3_2',
 'conv3_3',
 'pool3',
 'conv4_1',
 'conv4_2',
 'conv4_3',
 'pool4',
 'conv5_1',
 'conv5_2',
 'conv5_3',
 'pool5',
 'fc6',
 'fc7',
 'fc8',
 'prob']

ONNX 形式への変換には `onnx-chainer` パッケージを使用します。  
`onnx-chainer` は `pip` を使用してインストールすることができます。

In [ ]:
!pip install onnx-chainer

In [23]:
import onnx_chainer

Chainer は **Define-by-Run** というスタイルを採用しているため、ネットワーク構造は実際にデータを渡してフォワード（順伝播）計算を行うことで確定します。
（Define-by-Run は Chainer のコアコンセプトの１つであり、ネットワーク構造をフォワード計算をしながら定義することを指します。
このコンセプトによって、複雑なモデルをより少ないコード量で構築できる、デバックが容易になる、といったメリットがあります。）

ONNX 形式へ変換するためにはネットワーク構造が確定している必要があるため、ダミーの入力データを用意してフォワード計算を行います。
ダミーデータには、実際の推論に用いるデータと同じ `shape` の入力変数を使用します。
VGG16 モデルは 224×224 の3チャンネルの画像を入力データとして想定しているため、以下のようにダミーデータを作成します。

In [24]:
# ダミーデータの準備
import numpy as np
x = np.zeros((1, 3, 224, 224), dtype=np.float32)
x.shape

(1, 3, 224, 224)

Chainer を推論モードに切り替え、学習済みモデル `model` に対してダミーデータ `x` を渡してネットワーク構造を確定させた後、ONNX 形式のファイルとして出力します。

In [25]:
# 推論モードに切り替え
chainer.config.train = False

# ONNX 形式に変換する
onnx_model = onnx_chainer.export(model, x, filename='convnet.onnx')

In [1]:
# 出力されたファイルの確認
!ls convnet.onnx

convnet.onnx


これで、学習済みモデルを ONNX 形式のファイルに変換することができました。  

## ONNX 形式のモデルを Caffe2 で推論する

続けて、先ほど変換した ONNX 形式の学習済みモデルファイルを [Caffe2](https://caffe2.ai/) フレームワークで読み込み、推論を行います。

まず、Caffe2 とその依存モジュールをインストールします。

In [ ]:
# Caffe2 (PyTorch の一部) のインストール
!pip install torch

# Caffe2 の依存モジュールのインストール
!pip install onnx future

### Caffe2 に ONNX モデルを読み込む

Caffe2 に ONNX モデルを読み込みます。

In [28]:
# 保存された ONNX モデルの読み込み
import onnx
onnx_model = onnx.load('convnet.onnx')

### 読み込んだ ONNX モデルを使用して推論する

ここでは乱数データを入力にして推論を行います（実際には、入力として 224×224 の3チャンネル画像を使用します）。

In [29]:
# 推論の準備を行う
from caffe2.python.onnx import backend
prepared_backend = backend.prepare(onnx_model)

In [30]:
# ダミーの入力データを用意
x = np.random.randn(1, 3, 224, 224).astype(np.float32)

In [31]:
x.shape

(1, 3, 224, 224)

In [32]:
# 推論の実行
prepared_backend.run(x)

Outputs(Softmax_0=array([[2.14816275e-04, 1.08164025e-03, 4.11246932e-04, 8.18778411e-04,
        1.71566370e-03, 1.44439552e-03, 5.13035618e-03, 1.90243445e-04,
        1.06249157e-04, 1.54615147e-04, 4.11179295e-04, 2.20485352e-04,
        4.23722464e-04, 9.20125283e-04, 2.28955381e-04, 1.86870879e-04,
        3.95452895e-04, 1.71444262e-04, 4.76445537e-04, 4.93705564e-04,
        5.74271311e-04, 8.71971250e-04, 5.73325728e-04, 3.74813128e-04,
        1.26666389e-04, 8.78053252e-05, 6.43608335e-04, 3.97719909e-04,
        6.98198928e-05, 1.76480913e-03, 1.18460186e-04, 2.65453738e-04,
        1.50202672e-04, 1.38622615e-03, 2.58962763e-03, 3.39274702e-04,
        7.36668706e-04, 1.87365469e-04, 1.46729185e-03, 3.18095059e-04,
        7.21398508e-04, 3.76959128e-04, 4.75917652e-04, 3.02988483e-04,
        3.80057172e-04, 1.58852024e-03, 5.70400036e-04, 6.88131899e-04,
        8.37533094e-04, 7.47749116e-04, 1.19529001e-03, 2.85340007e-04,
        8.71164957e-04, 1.51232968e-03, 6.6560

### ONNX 形式のモデルを Caffe2 形式に変換する

前節では Chainer の学習済みモデルを ONNX 形式に変換し、Caffe2 を使用してそのモデルで推論を行いました。
本節では、ONNX 形式のモデルを Caffe2 形式に変換する方法を説明します。
Caffe2 はモバイル環境に対応しているため、Chainer で作成した学習済みモデルを ONNX 形式を経由して Caffe2 形式に変換することで、モバイルアプリへの組み込みが可能になります。

In [34]:
# ONNX モデルを Caffe2 モデルに変換
init_net, predict_net = backend.Caffe2Backend.onnx_graph_to_caffe2_net(onnx_model, device='CPU')

In [36]:
# Caffe2 モデルの保存
from caffe2.python.onnx.helper import save_caffe2_net
save_caffe2_net(init_net , 'caffemodel_init.pb', output_txt=False)
save_caffe2_net(predict_net , 'caffemodel_predict.pb', output_txt=False)

In [2]:
# 出力されたファイルの確認
!ls *.pb

caffemodel_init.pb  caffemodel_predict.pb


## Menoh を使用した他言語での推論

前節では、Chainer で学習したモデルを、ONNX を介して異なるフレームワークで推論する方法をお伝えしました。
本節では ONNX 形式のモデルを読み込んで推論を行う [Menoh](https://github.com/pfnet-research/menoh)（メノウ）フレームワークの方法を解説します。

Menoh は ONNX 形式で保存された学習済みモデルを読み込むことのできる、推論専用の深層学習フレームワークです。
Menoh は、C/C++ で実装された Menoh 本体（コアライブラリ）と各言語向けのバインディングから構成されており、様々なプログラミング言語から呼び出せるように設計されていることが大きな特徴の一つです。
2019年2月現在、C, C++, C#, Ruby, Java, Go, Node.js, Haskell, Rust から Menoh を利用することができます。

ここでは、Ruby から Menoh を使用して推論を行う方法を紹介します。

### Menoh-Ruby の環境構築

#### Ruby のインストール

In [ ]:
!apt install ruby ruby-dev

#### Menoh のインストール

[Menoh](https://github.com/pfnet-research/menoh) 本体のインストールを行います。
なお、この環境構築の手順は Ubuntu 18.04（Google Colaboratory 環境）を想定しています。
それ以外の環境でのセットアップ手順は [README](https://github.com/pfnet-research/menoh#installation-using-package-manager-or-binary-packages) を参照してください。

In [ ]:
!curl -LO "https://github.com/pfnet-research/menoh/releases/download/v1.1.1/ubuntu1804_mkl-dnn_0.16-1_amd64.deb"
!curl -LO "https://github.com/pfnet-research/menoh/releases/download/v1.1.1/ubuntu1804_menoh_1.1.1-1_amd64.deb"
!curl -LO "https://github.com/pfnet-research/menoh/releases/download/v1.1.1/ubuntu1804_menoh-dev_1.1.1-1_amd64.deb"
!apt install ./ubuntu1804_*_amd64.deb

#### Menoh-Ruby のインストール

Menoh の [Ruby バインディング](https://github.com/pfnet-research/menoh-ruby)のインストールを行います。

In [ ]:
!gem install menoh

###  Ruby で推論を行う

Menoh-Ruby の[サンプルスクリプト](https://github.com/pfnet-research/menoh-ruby/blob/3e0a972dcfcf3c4af7a681b6400ae47f85cbd383/example/example_vgg16.rb)を `example_vgg16.rb` という名前で保存して、実行してみましょう。
ONNX 形式の学習済みモデルファイル（VGG16）とテスト用の画像2枚が `data` ディレクトリにダウンロードされた後、推論が実行されます。

In [ ]:
# サンプルスクリプトで使用するライブラリのインストール
!apt install libmagickwand-dev
!gem install rmagick

In [ ]:
# サンプルスクリプトのダウンロードと実行
!curl -LO "https://raw.githubusercontent.com/pfnet-research/menoh-ruby/3e0a972dcfcf3c4af7a681b6400ae47f85cbd383/example/example_vgg16.rb"
!mkdir -p data/
!ruby "example_vgg16.rb"

推論結果が以下のように表示されれば成功です。

```
=== Result for ./data/Light_sussex_hen.jpg ===
n01514859 hen : 0.9339964389801025
n01514668 cock : 0.05969797447323799
n01807496 partridge : 0.002985152183100581
n01797886 ruffed grouse, partridge, Bonasa umbellus : 0.0008867944125086069
n01847000 drake : 0.0005284951184876263

=== Result for ./data/honda_nsx.jpg ===
n04285008 sports car, sport car : 0.6823536157608032
n04037443 racer, race car, racing car : 0.1251951903104782
n03100240 convertible : 0.11076415330171585
n02974003 car wheel : 0.047828804701566696
n03459775 grille, radiator grille : 0.009155559353530407
```

#### サンプルスクリプトの内容について

ここでは、[サンプルスクリプト](https://github.com/pfnet-research/menoh-ruby/blob/3e0a972dcfcf3c4af7a681b6400ae47f85cbd383/example/example_vgg16.rb)の概要について説明します。
（モデルや画像のデータセットのダウンロードは省略します。）  

最初に、サンプルスクリプトで使用する各ライブラリの読み込みを行います。
Ruby で画像を扱うライブラリとして [RMagick](https://github.com/rmagick/rmagick) を使用しています。

```ruby
require 'open-uri'
require 'rmagick'
require 'menoh'
```

ONNX 形式のモデルデータと推論に使うサンプルデータをダウンロードします。

```ruby
# download dependencies
...
```

推論に使うサンプルデータのファイル名と、学習モデル（VGG16）の入力データ形式を定義します。
`rgb_offset` は学習済みの

```ruby
# load dataset
image_list = [
  './data/Light_sussex_hen.jpg',
  './data/honda_nsx.jpg'
]
input_shape = {
  channel_num: 3,
  width: 224,
  height: 224
}
rgb_offset = {
  R: 123.68,
  G: 116.779,
  B: 103.939
}
```

Menoh を使用して ONNX 形式のモデルデータを読み込みます。

```ruby
# load ONNX file
onnx_obj = Menoh::Menoh.new './data/VGG16.onnx'
```

ONNX 形式のモデルでは、ニューラルネットワークの各部分（ノード）にユニークな ID が振られています。
Menoh ではネットワークの任意の部分に対してデータを入出力することができるため、データの入出力対象となる ID を指定する必要があります。
ネットワークの各部分に設定された ID は、モデルファイルを [Netron](https://github.com/lutzroeder/Netron) などの可視化ツールで読み込むことで確認できます。

* `CONV1_1_IN_NAME` は入力層（最初の畳み込み層）の ID で、この層にデータを入力します。
* `FC6_OUT_NAME` は全結合層（1番目）の ID です。本スクリプトでは、この層の出力結果は使用しません（複数の出力を指定するサンプルコードを示す目的で定義されています）。
* `SOFTMAX_OUT_NAME` は出力層（最終層の出力に Softmax 関数を適用したもの）の ID で、推論の結果（各分類クラスに対するスコア）を得ることができます。

```ruby
# onnx variable name
CONV1_1_IN_NAME = 'Input_0'.freeze
FC6_OUT_NAME = 'Gemm_0'.freeze
SOFTMAX_OUT_NAME = 'Softmax_0'.freeze
```

読み込んだ ONNX 形式のモデルをどのように実行するかを定義し、「Menoh モデル」を作成します。
ここではバックエンドとして `mkldnn` （Intel CPU 向けのハードウェアアクセラレーション）を使用します。
また、入力層の ID とそのデータ形式、出力層の ID を定義します。

```ruby
# model options for model
model_opt = {
  backend: 'mkldnn',
  input_layers: [
    {
      name: CONV1_1_IN_NAME,
      dims: [
        image_list.length,
        input_shape[:channel_num],
        input_shape[:height],
        input_shape[:width],
      ]
    }
  ],
  output_layers: [FC6_OUT_NAME, SOFTMAX_OUT_NAME]
}

# make model for inference under 'model_opt'
model = onnx_obj.make_model model_opt
```

推論対象の画像を読み込み、データセットの準備を行います。

1. `image_list` から順番に画像ファイルのパスを読み込み、`image_filepath` に格納します。
2. RMagick で画像を読み込みます（画像の読み込み後に Array の形式となるため、`.first` で最初の要素を取得します）。
3. 読み込んだ画像を VGG16 の入力サイズにあわせてリサイズします。
4. RGB の各ピクセル値を 16-bit から 8-bit に変換（256 で割る）し、中間画像の offset の値を引く正規化を行います。

```ruby
# prepare dataset
image_set = [
  {
    name: CONV1_1_IN_NAME,
    data: image_list.map do |image_filepath|  # 1
      image = Magick::Image.read(image_filepath).first  # 2
      image = image.resize_to_fill(input_shape[:width], input_shape[:height])  # 3
      'RGB'.split('').map do |color|
        image.export_pixels(0, 0, image.columns, image.rows, color).map do |pix|
          pix / 256 - rgb_offset[color.to_sym]  # 4
        end
      end.flatten
    end.flatten
  }
]
```

推論を実行します。

```ruby
# execute inference
inferenced_results = model.run image_set
```

推論結果を、対応するカテゴリ名と共に出力します。

```ruby
# load category definition
categories = File.read('./data/synset_words.txt').split("\n")
TOP_K = 5
layer_result = inferenced_results.find { |x| x[:name] == SOFTMAX_OUT_NAME }
layer_result[:data].zip(image_list).each do |image_result, image_filepath|
  puts "=== Result for #{image_filepath} ==="

  # sort by score
  sorted_result = image_result.zip(categories).sort_by { |x| -x[0] }

  # display result
  sorted_result[0, TOP_K].each do |score, category|
    puts "#{category} : #{score}"
  end
end
```

本章では、Chainer で作成した学習済みモデルを、様々なフレームワークや言語で推論する方法を解説しました。
これにより、学習済みモデルを様々な環境で使用したり、アプリケーションに組み込むことが可能となります。